In [21]:
from pdf_gene import MoonCalc ,generate_pdf
from datetime import datetime
import requests
from tqdm import tqdm
import pandas as pd
from google import genai
import os
from dotenv import load_dotenv
import json
import markdown
import pdfkit
from markdown_pdf import MarkdownPdf, Section


In [3]:
# Path to your .env file (outside the project folder)
load_dotenv(dotenv_path='../.env')

api_key = os.getenv("GEMINI_API_KEY")

In [4]:
def final_df(date,month, year, path="D:/code/Data_2035", dst='D:/Output'):
    
    date_obj = datetime.strptime(date, "%d-%m-%Y")    
    # Convert to YYYY-MM-DD format
    converted_date = date_obj.strftime("%Y-%m-%d")
    Moon = MoonCalc(path,converted_date,month,year +" AH",dst)
    return Moon.calculate()
final_df = final_df("28-04-2025","ZULQADDAH","1446")

In [5]:
df = final_df
# Split 'STATION(Sunset)' into 'Station' and 'SunsetTime'
df[['Station', 'SunsetTime']] = df['STATION(Sunset)'].str.extract(r'^(.*?)\s*\((.*?)\)$')

# Optional: drop the original column if no longer needed
df.drop(columns=['STATION(Sunset)'], inplace=True)

# Reorder columns if desired
df = df[['Station', 'SunsetTime', 'LAG TIME(Minutes)', 'MOON ALTITUDE(Degrees)',
         'SUN_AZIMUTH(Degrees)', 'DAZ(Degrees)', 'ELONGATION(Degrees)', 
         'ILLUMINATION(%)', 'CRITERION']]

row = df
row.columns = [
    'Station', 'SunsetTime', 'LagTime', 'MoonAltitude', 'SunAzimuth',
    'DAZ', 'Elongation', 'Illumination', 'Criterion'
]


In [6]:
visibility_mapping = {
    'A': 'Easily visible',
    'B': 'Visible under perfect conditions',
    'C': 'May need optical aid to find the crescent Moon',
    'D': 'Will need optical aid to find the crescent Moon',
    'E': 'Not visible with a telescope',
    'F': 'Not visible, below the Danjon limit'
}


In [9]:
import json
import re

def sanitize_and_parse_json(text):
    try:
        # Try parsing directly first
        return json.loads(text)
    except json.JSONDecodeError:
        # If direct parsing fails, try extracting JSON manually
        match = re.search(r'\{.*\}', text, re.DOTALL)
        if match:
            try:
                cleaned_text = match.group()
                return json.loads(cleaned_text)
            except json.JSONDecodeError:
                pass
        return None



In [10]:
def make_prompt(row):
    # Visibility criterion description
    visibility_description = visibility_mapping.get(row['Criterion'], 'Unknown visibility criterion')

    # Interpret illumination
    illumination = float(row['Illumination'])
    illumination_description = (
        "The illumination is high, indicating good visibility."
        if illumination > 0.8 else
        "The illumination is low, which might make the Moon harder to see."
    )

    # Interpret moon altitude
    moon_altitude = float(row['MoonAltitude'])
    altitude_description = (
        "The Moon is high above the horizon, aiding visibility."
        if moon_altitude > 8 else
        "The Moon is low on the horizon, which may limit visibility."
    )

    return (
        "Here is the required format:\n"
        "{\n"
        "  \"visibility\": \"...\",\n"
        "  \"Explanation\": \"...\",\n"
        "  \"Confidence\": \"...\",\n"
        "  \"Conclusion\": \"...\"\n"
        "}\n"
        "Answer based on the following data:\n"
        f"On {row.name}, at station {row['Station']}, "
        f"at the time of Sunset {row["SunsetTime"]},"
        f"moon altitude {moon_altitude}°, "
        f"illumination {illumination}%, ",
        f"and visibility criterion '{row['Criterion']}': {visibility_description}. ",
        f"{illumination_description} {altitude_description} ,"
        f"Based on this data, explain whether the moon is likely to be visible and why.",
        f"Rate your confidence in your answer on a scale of 1-10 and explain briefly.",
#         f"Translate this response into Urdu."


    )

def query_gemini(prompt):

    client = genai.Client(api_key=api_key)

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt
    )
    return(response.text)

# ✅ Function to query LLaMA 3.2 via Ollama
def query_ollama(prompt, model="llama3.2"):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": model, "prompt": prompt, "stream": False}
    )
    return response.json()["response"].strip()

tqdm.pandas()  # This enables tqdm integration with pandas

df['llama_response'] = df.progress_apply(lambda row: query_gemini(make_prompt(row)), axis=1)
print(df[['Station', 'llama_response']].head())



100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:25<00:00,  1.96s/it]

              Station                                     llama_response
date                                                                    
2025-04-28     Cherat  ```json\n{\n  "visibility": "Likely Visible",\...
2025-04-28        Dir  ```json\n{\n  "visibility": "Easily visible",\...
2025-04-28     Gilgit  ```json\n{\n  "visibility": "Likely visible",\...
2025-04-28  Islamabad  ```json\n{\n  "visibility": "Easily visible",\...
2025-04-28     Jiwani  ```json\n{\n  "visibility": "Visible",\n  "Exp...



C:\Users\Administrator\AppData\Local\Temp\ipykernel_19892\1926361563.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['llama_response'] = df.progress_apply(lambda row: query_gemini(make_prompt(row)), axis=1)


In [11]:
df["llama_response"]
text = []
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for content in df["llama_response"]:
        response_text = content   # from Gemini
#         print(response_text)
        parsed_data = sanitize_and_parse_json(response_text)

        if parsed_data:
            text.append(parsed_data["Explanation"])
#             print("---------------x-xxxxxxxxxxxxxxx------------")
        else:
            print("Invalid JSON after cleaning")


In [12]:
explanations_list = text  # your collected explanations

prompt = f"""
You are provided with a list of explanations about the moon's visibility. Analyze them carefully.

{explanations_list}

Task:
- Summarize the key factors influencing visibility.
- Identify common challenges or favorable conditions.
- Write a structured report of findings.
"""

client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[prompt]
)

In [20]:
# Create a MarkdownPdf object. Set TOC level to 2 and optimize the output
pdf = MarkdownPdf(toc_level=1, optimize=True)

# Ensure the first section starts with a top-level header, which is required for TOC
# pdf.add_section(Section("# Moon Visibility Report\n", toc=False))

# Add another section with links and content
text =response.text
pdf.add_section(Section(text))



# Set document metadata like title and author
pdf.meta["title"] = "Moon Visibility Report"
pdf.meta["author"] = "Your Name"

# Save the generated PDF
pdf.save("moon_visibility_report.pdf")
